# Symmetric crosscuts and tropical moduli

by [Dominic Bunnett](http://page.math.tu-berlin.de/~bunnett) and [Michael Joswig](http://page.math.tu-berlin.de/~joswig/)

This is the software companion to the paper with the same name.  Its purpose is a computational proof for the following.

**Theorem.**
The stacky fan ${\mathbb M}_{K_4}^{\text{pl}}$ of tropical honeycomb curves has face vector $(7,18,25,21,8)$, and its link has trivial integral homology.

The computation has been tested with [polymake](https://www.polymake.org), version 4.3 (of 15 Dec 2020).

This computation is ad hoc, for this particular example.  Future versions of polymake will support a more sophisticated implementation of stacky fans.  We can measure the excution time for the entire notebook.

In [1]:
use Benchmark qw(:all);
$t0 = Benchmark->new;

In [2]:
application "fan";

We are interested in (quartic) tropical curves of fixed genus $g=3$.  The moduli space is $6$-dimensional, corresponding to the six edges of a trivalent graph on four nodes.

We pick labels $u,v,w,x,y,z$ for the edges and the corresponding standard basis vectors in ${\mathbb R}^6$. 

In [3]:
($u, $v, $w, $x, $y, $z) = 0..5;
$UnitMatrix = unit_matrix(6);
($U, $V, $W, $X, $Y, $Z) = map { new Vector($UnitMatrix->[$_]) } (0..5);
$Orthant=new Cone(RAYS=>$UnitMatrix);

By Theorem 5.1 of [Brodsky et al. (2015)](https://link.springer.com/article/10.1186/s40687-014-0018-1) a complete graph $K_4$ arises from a smooth tropical quartic (also called *honeycomb curve*) if and only if its edge lengths satisfy the following linear inequalities; they cut out a polyhedral cone in the $6$-dimensional orthant of all $K_4$-curves.

In [4]:
$D_ineqs = new Matrix( [$U-$V, $V-$W, $W-$Y, $W-$Z, $V-$X] );
$D_cone = new Cone(INEQUALITIES=>$D_ineqs/$UnitMatrix);

print $D_cone->F_VECTOR;

11 31 39 25 8

Click here for additional output 
 
polymake: used package ppl
 The Parma Polyhedra Library ([[wiki:external_software#PPL]]): A C++ library for convex polyhedra
 and other numerical abstractions.
 http://www.cs.unipr.it/ppl/

That cone is the fundamental domain $D$ constructed in Section 5.2 of the article.  It forms one sixth of the cone descibed in Theorem 5.1 of [Brodsky et al. (2015)](https://link.springer.com/article/10.1186/s40687-014-0018-1).  The next computation yields the volume of the link; the number $4/15$ in Table on p.18 of [Brodsky et al. (2015)](https://link.springer.com/article/10.1186/s40687-014-0018-1) arises as four times this value.

In [5]:
$D_link = new Polytope($D_cone);
print $D_link->VOLUME;

1/15

The graph $K_4$ forms the skeleton of these tropical curves, and we are interested in the action of the symmetric group of degree four on the six edges.

In [6]:
$skeleton = new IncidenceMatrix([[$u,$v,$x],[$v,$w,$z],[$u,$w,$y],[$x,$y,$z]]);
$group = group::automorphism_group($skeleton,0);

print $group->ORDER;

24

Click here for additional output 
 
polymake: used package nauty
 Computation of automorphism groups of graphs.
 Copyright by Brendan McKay and Adolfo Piperno.
 http://pallini.di.uniroma1.it/
polymake: used package permlib
 A callable C++ library for permutation computations. 
 Written by Thomas Rehn.
 http://www.math.uni-rostock.de/~rehn/software/permlib.html

Now we produce code which works for any cone which is properly refined by the braid arrangement.

Our first function computes the cone corresponding to a (possibly low-dimensional) cell of the chamber complex of a hyperplane arrangement.  This may be called often for the same arguments; so we cache it.

In [7]:
%realizationcache = ();

sub get_realization{
   my ($C, # Cone
       $index # index to face in Hasse diagram of chamber complex
       ) = @_;
   my $set = $C->HASSE_DIAGRAM->FACES->[$index];
   my $R = $C->RAYS;
   my $L = $C->LINEALITY_SPACE;
   if(!$realizationcache{$index}){
      $realizationcache{$index} = new Cone(INPUT_RAYS=>$R->minor($set,All),INPUT_LINEALITY=>$L);
   }
   return $realizationcache{$index};
}

The next function computes the sweep of a cone corresponding to a (possibly low-dimensional) cell of the chamber complex of a hyperplane arrangement.  Again this may be called often for the same arguments; so we cache it.

Attention: the caches are global variables.

In [8]:
%conesweepcache = (); # key: Int, value: Set<Matrix>

sub cone_sweep {
   my ($C,  # Cone
       $index, # index to face in Hasse diagram of chamber complex; this is the cone "C"
       $G      # group acting on coordinate directions of C, by permutations
       )=@_; 
   if(!defined $conesweepcache{$index}){
      $conesweepcache{$index} = cone_sweep_inner($C, $index, $G);
   }
   return $conesweepcache{$index}; # Set<Matrix>
}

sub cone_sweep_inner {
  my ($inputC,$index,$G)=@_; # group G acting on coordinate directions of cone C, by permutations
  my $C = get_realization($inputC,$index);
  my $F = new Matrix($C->FACETS / $C->LINEAR_SPAN / -$C->LINEAR_SPAN);
  my $o = group::orbit<group::on_cols>($G->PERMUTATION_ACTION->GENERATORS, $F);
  my @o = map { new Cone(INEQUALITIES=>$_) } @{$o};
  my $o2 = new Set<Matrix>(map(new Matrix(sort @{$_->RAYS}), @o));
  return $o2;
}

Since the elements in a set are sorted.  Picking the first in an orbit makes for a canonical choice. 

In [9]:
sub canonical_representative {
   my ($C, $index, $G)=@_; # group G acting on coordinate directions of cone C, by permutations
   my $sweep = cone_sweep($C, $index, $G);
   return $sweep->front(); # of type Matrix
}

Extracts group orbits and representatives for a given dimension.

In [10]:
$map_to_rep = new Map<Int,Int>; # maps each face to its orbit rep (global)
$top_node = $D_cone->HASSE_DIAGRAM->TOP_NODE;
$map_to_rep->{$top_node} = $top_node;

sub extract_orbit_reps {
   my ($C, # cone
       $k,    # dimension
       $G     # group
       ) = @_;
   my @in = @{$C->HASSE_DIAGRAM->nodes_of_dim($k)};
   my $orbits = new Set<Set>;
   while(@in > 0){
      my $first = pop @in;
      $map_to_rep->{$first} = $first;
      my $this_orbit = new Set($first);
      my $cr = canonical_representative($C, $first, $G);
      @in = grep{
            my $other_cr = canonical_representative($C, $_, $G);
            my $reps_are_distinct = $other_cr != $cr;
            if (!$reps_are_distinct) { 
                $this_orbit += $_;
                $map_to_rep->{$_} = $first;
            };
            $reps_are_distinct;
      } @in;
      $orbits += $this_orbit;
   }
   return $orbits; # of type (Set, Set<Set>)
}

We collect all data of faces needed.

In [11]:
sub all_orbits {
  my ($C, # cone
      $G     # group
      ) = @_;
  my $full_orbits = new Array<Set<Set>>(6);
  for (my $k=0; $k<6; ++$k) {
    my $orbits = extract_orbit_reps($C, $k+1, $G);
    $full_orbits->[$k] = $orbits;
  }
  return $full_orbits; # of type Array<Set<Set>>
}

In [12]:
$honey_orbits = all_orbits($D_cone, $group);

The face vector of the stacky fan.

In [13]:
print $honey_orbits->[$_]->size(), " " for 0..5; 

7 18 25 21 8 1 

# We now barycentrically subdivide in order to compute the integral homology

First thing we do is pass to the barycentric subdivision.


In [14]:
$HD = $D_cone->HASSE_DIAGRAM;
$bsd = new Set<Set>;
$cnt = 0;

for (my $i6 = entire($HD->nodes_of_dim(6)); $i6; ++$i6) {
   for (my $i5 = entire($HD->ADJACENCY->in_adjacent_nodes($$i6)); $i5; ++$i5) {
      for (my $i4 = entire($HD->ADJACENCY->in_adjacent_nodes($$i5)); $i4; ++$i4) {
         for (my $i3 = entire($HD->ADJACENCY->in_adjacent_nodes($$i4)); $i3; ++$i3) {
            for (my $i2 = entire($HD->ADJACENCY->in_adjacent_nodes($$i3)); $i2; ++$i2) {
               for (my $i1 = entire($HD->ADJACENCY->in_adjacent_nodes($$i2)); $i1; ++$i1) {
                  my $b_face = new Set($$i6,
                  $map_to_rep->{$$i5},
                  $map_to_rep->{$$i4},
                  $map_to_rep->{$$i3},
                  $map_to_rep->{$$i2},
                  $map_to_rep->{$$i1},
                  );
                  $bsd += $b_face;
                  ++$cnt;
               }
            }
         }
      }
   }
}

We call XX the barycentric subdivision of the honeycomb locus.

In [15]:
$bsd = maximal_chains_of_lattice($HD)->minor(All,~[$HD->BOTTOM_NODE]);

$XX = new topaz::SimplicialComplex(INPUT_FACES=>rows($bsd));

print
   $XX->FACETS->[5], "\n",
   $XX->F_VECTOR, " = ", sum($XX->F_VECTOR);

{0 4 22 52 94 104}
115 1218 4400 7136 5376 1536 = 19781

We then pass to the second barycentric subdivision which we call YY $=Y$.

The numbers now start to get bigger...

In [16]:
$YY = topaz::barycentric_subdivision($XX);

print
  $YY->FACETS->[5], "\n",
  $YY->N_FACETS, "\n",
  $YY->F_VECTOR, " = ", sum($YY->F_VECTOR);

{352 1644 6912 14048 18448 19666}
1105920
19781 385252 1919136 3857664 3409920 1105920 = 10697673

We define a function mapping a set of faces to the set consisting of only representatives.
Essentially generalising the map_to_rep function.

In [17]:
sub map_set_to_reps {
  my ($set_of_faces_of_honey_cone) = @_; # Set
  my @LL = map { $map_to_rep->{$_} } @{ $set_of_faces_of_honey_cone };
  return new Set(@LL); # does not need to be a flag of honey_cone, even if the input is
}

We apply this function to $Y$.

In [18]:
$n_vertices_bsd2 = $YY->N_VERTICES;
$L = new Array<Set>($n_vertices_bsd2);
$string_to_id = new Map<String,Int>;

for (my $i=0; $i<$n_vertices_bsd2; ++$i) {
  my $this_set = new Set($YY->VERTEX_LABELS->[$i]); 
  my $mapped_set = map_set_to_reps($this_set); 
  $string_to_id->{"$mapped_set"} = $i unless exists( $string_to_id->{"$this_set"} ); # pick first as rep
  $L->[$i] = $mapped_set; 
}

Vertices of $Y$, i.e., faces of $X$ are mapped to sets of representatives; also the data types differ.

In [19]:
$k=5; # for example
$s = map_set_to_reps( new Set($YY->VERTEX_LABELS->[$k]) );

print
  $YY->VERTEX_LABELS->[$k], " ",
  $YY->VERTEX_LABELS->type->full_name, "\n",
  $L->[$k], " ",
  $L->type->full_name, "\n",
  $string_to_id->{"$s"};

{0 4 22 52 94 104} Array<String>
{0 4 22 56 94 104} Array<Set<Int>>
5

Iterate over all facets of $Y$, i.e., maximal flags of $X$.

In [20]:
$n_facets_bsd2 = $YY->N_FACETS;
$identified = new Array<Set>($n_facets_bsd2);

for (my $i=0; $i<$n_facets_bsd2; ++$i) {
  my $facet_of_projection = new Set;
  for (my $v=entire($YY->FACETS->[$i]); $v; ++$v) { # v is a face of X, i.e., a not necessarily maximal flag of D
    my $vertex_of_Y_as_set_of_faces_of_D = $L->[$$v];
    my $LL_as_set = map_set_to_reps($vertex_of_Y_as_set_of_faces_of_D);
    my $vertex_of_projection = $string_to_id->{"$LL_as_set"};
    $facet_of_projection += $vertex_of_projection;
  }
  $identified->[$i] = $facet_of_projection;
}

We define $Z$ to be the moduli space of honeycomb curves. Since we have passed to the second barycentric subdivision, $Z$ can be given explicitly as a simplicial complex.

In [21]:
$ZZ = new topaz::SimplicialComplex(INPUT_FACES=>$identified);

In [22]:
print $ZZ->F_VECTOR, " = ", sum($ZZ->F_VECTOR);

14281 281790 1413750 2855280 2532720 823680 = 7921501

Finally, we can compute that $Z$ has trivial (reduced) homology.

In [23]:
print $ZZ->HOMOLOGY;

({} 0)
({} 0)
({} 0)
({} 0)
({} 0)
({} 0)


This finishes the proof.

In [24]:
$t1 = Benchmark->new;
$td1=timediff($t1,$t0);
print "total time ", timestr($td1);

total time 499 wallclock secs (498.66 usr +  1.55 sys = 500.21 CPU)

On AMD Ryzen 9 3950X running Manjaro/Linux with kernel 5.10.2-2.